# <center>Colorado K-12 Financial Transparency – <br>An Exploratory Data Science Comparison Across Schools</center>

### <center>MSDS 696: Data Science Practicum II - Stacey Sandy</center>

### <u><b>Overview </b></u>
This project is for the fulfillment of my final course in the pursuit of my Master of Science in Data Science degree at Regis University. In this past academic year, my kindergartner son has brought home 5 different fundraisers and a quarterly scholastic book fair flyer. I understand the scholastic book fair awards books to classes that purchase the most books. Unfortunately, I am unclear where the multiple fundraiser funding is going, what the funding is for, and how fundraisers feed into the school. These implications on the school and district fundraiser efforts need further analysis. Especially since school ends in a little over two months, I am curious to learn how expenses are handled. I intend to compare expenses across schools and then across districts. I would like to analyze which schools are spending more than others and why their expenses are more or less than others. I want to learn the cause and effects as to what are the trends on expenditures. Hopefully, I could learn more about why fundraising is important or necessary multiple times a year for any school. 

### <u><b>Objective</b></u>
The objective of this project and the primary research question is to address how financial expenditures are being spent across K through 12 schools in Colorado. An exploratory data science approach will be conducted to answer this research question. Overall, what are the school expenses that impact our children’s educational needs? Could the type of staffing, teacher types, and number of students be an impact? What are the contributing factors of the various expenses? How do schools and districts compare to one another? What variables are impacting the data reported? Could federal and state funding not be enough? 

### <u><b>Data Source</b></u>
* Colorado Department of Education website on Colorado’s K12 Financial Transparency website: https://coloradok12financialtransparency.com/#/
* The complete dataset of all Colorado K12 districts and BOCES can also be downloaded at: https://s3.amazonaws.com/bb-pub-pipeline-production/home/circleci/repo/public/uploads/data_files/uploads/000/000/416/20200114030354-colorado-financial-public-data-pipeline-v-1-Run_1-export-source-lep-facts.csv20200114-2678-rqjqmn.csv?1578999834

### <u><b>Financial Policies and Procedures Documentation</b></u>
* Colorado Department of Education. (2016, July 1). <i>Financial Policies and Procedures Chart of Accounts.</i> Retrieved from https://www.cde.state.co.us/cdefinance/fpp_coa1617

#### The following website is available for direct school or district searchs within Colorado:
* Colorado K12 Financial Transparency website: https://coloradok12financialtransparency.com/#/

### Project Process:<br>
<br>
1. Conduct exploratory data analysis on complete dataset of all Colorado K12 districts and BOCES.<br>
2. Investigate unique Ids, values, and row/column data.<br>
3. Highlights financial codes that are max, min, mean, and of interest.<br>
4. Add additional data from Colorado Department of Education on financial chart of accounts.<br>
5. <br>
<br>
In final, compare and contrast findings. Answer research questions. Did you accomplish what your project objective detailed?

## <center>Exploratory Data Analysis (EDA) on all Colorado K12 districts and BOCES</center>

In [1]:
# Load libraries
import pandas as pd
from pandas_summary import DataFrameSummary
import numpy as np
from pandas_profiling import ProfileReport
import multiprocessing
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import json

In [2]:
# Load the CO K12 Dataset into a dataframe.
COK12_df = pd.read_csv('Data/colorado-financial-public-data-pipeline-v-1-Run#1-export-source-lep-facts.csv', header=4)

In [3]:
# Ensure columns were retrieved
COK12_df.columns

Index(['district_code', 'admin_unit', 'school_code', 'fund_code',
       'location_code', 'sre_code', 'program_code', 'object_source_code',
       'job_class_code', 'grant_code', 'amount_cents', 'exclude_school',
       'exclude_district'],
      dtype='object')

In [4]:
# Look at dataset head
COK12_df.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
0,3080,64203,0,10,600,0,20,100,200,0,71250.0,f,f
1,3080,64203,0,10,600,0,10,100,200,4010,13841274.0,f,f
2,3080,64203,0,10,700,0,2700,100,100,0,12171639.0,f,f
3,3080,64203,0,21,700,0,3100,100,100,0,4079700.0,f,f
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925.0,f,f


In [5]:
# Look at dataset tail
COK12_df.tail()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
280681,1180,64233,570,10,300,0,1800,810,0,0,340700.0,f,f
280682,1180,64233,429,11,951,0,2500,810,0,0,-700.0,f,f
280683,1180,64233,429,11,951,0,2600,810,0,0,45477.0,f,f
280684,1180,64233,429,11,951,0,2800,810,0,0,24500.0,f,f
280685,1180,64233,429,11,951,0,18,810,0,0,2499.0,f,f


In [6]:
# View dataframe shape (# of Rows and # of Columns)
COK12_df.shape

(280686, 13)

In [7]:
# Look at dataset information details
COK12_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280686 entries, 0 to 280685
Data columns (total 13 columns):
district_code         280686 non-null int64
admin_unit            280686 non-null int64
school_code           280686 non-null int64
fund_code             280686 non-null int64
location_code         280686 non-null int64
sre_code              280686 non-null int64
program_code          280686 non-null int64
object_source_code    280686 non-null int64
job_class_code        280686 non-null int64
grant_code            280686 non-null int64
amount_cents          280686 non-null float64
exclude_school        280686 non-null object
exclude_district      280686 non-null object
dtypes: float64(1), int64(10), object(2)
memory usage: 27.8+ MB


In [8]:
#Let's confirm a count on null valuws in the entire data set.
print(np.count_nonzero(COK12_df.isnull()))

0


In [9]:
# Look at the data frame summary data
COK12_dfs = DataFrameSummary(COK12_df)

In [10]:
COK12_dfs.columns_stats

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents,exclude_school,exclude_district
counts,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686,280686
uniques,200,62,1920,34,258,45,317,415,94,228,222337,2,2
missing,0,0,0,0,0,0,0,0,0,0,0,0,0
missing_perc,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
types,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,numeric,bool,bool


In [11]:
# Let's make a copy of the dataframe.
COK12_df2 = COK12_df.copy()

### Remember we must take note of the following information provided within the dataset (csv file)
<br>
When summarizing this information there are three important points to keep in mind:<br>
1) Fund 90 is used for informational purposes only and must be excluded from any analysis.<br>
2) All amounts are displayed with cents and without decimal places. E.g. the value $5.42 will be recorded as 542. <br>
3) The data methodology page contains details on the categories excluded from this summary at both the District and the School level.

In [12]:
# Check matplotlib version (should be 3.1.2)
matplotlib.__version__

'3.1.2'

In [18]:
#Use Anaconda Prompt to check version (should be 3.7.4)
python.__version__

NameError: name 'python' is not defined

In [14]:
# Drop the exclude_school and exclude_district columns from data frame
COK12_df2 = COK12_df2.drop(['exclude_school', 'exclude_district'],axis=1)

In [15]:
# Check columns again
COK12_df2.columns

Index(['district_code', 'admin_unit', 'school_code', 'fund_code',
       'location_code', 'sre_code', 'program_code', 'object_source_code',
       'job_class_code', 'grant_code', 'amount_cents'],
      dtype='object')

In [16]:
# Look at data frame head
COK12_df2.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250.0
1,3080,64203,0,10,600,0,10,100,200,4010,13841274.0
2,3080,64203,0,10,700,0,2700,100,100,0,12171639.0
3,3080,64203,0,21,700,0,3100,100,100,0,4079700.0
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925.0


In [19]:
# Change ammount_cents to int64 not float64
COK12_df2 = COK12_df2.astype({'amount_cents':'int64'})

In [20]:
COK12_df2.dtypes

district_code         int64
admin_unit            int64
school_code           int64
fund_code             int64
location_code         int64
sre_code              int64
program_code          int64
object_source_code    int64
job_class_code        int64
grant_code            int64
amount_cents          int64
dtype: object

In [21]:
# Look at tail and ensure amount_cents is no longer showing as a float type.
COK12_df2.tail()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
280681,1180,64233,570,10,300,0,1800,810,0,0,340700
280682,1180,64233,429,11,951,0,2500,810,0,0,-700
280683,1180,64233,429,11,951,0,2600,810,0,0,45477
280684,1180,64233,429,11,951,0,2800,810,0,0,24500
280685,1180,64233,429,11,951,0,18,810,0,0,2499


In [43]:
#Let's count how many different financial values are in each column.
COK12_districtCodes = pd.value_counts(COK12_df2['district_code'].values,sort=True)
COK12_districtCodes.head()

880     34937
1420    21133
900     15183
20      11193
1550    10629
dtype: int64

### Investigate the maximum  top 5 district_codes!?!

In [23]:
COK12_districtCodes.tail()

9140    71
9165    56
8042    42
8041    28
9120    22
dtype: int64

In [44]:
#Let's count how many different financial values are in each column.
COK12_adminUnits = pd.value_counts(COK12_df2['admin_unit'].values,sort=True)
COK12_adminUnits.head()

16010    35049
30011    21133
18010    15183
1020     11193
35010    10629
dtype: int64

### Investigate the maximum  top 5 admin_unit codes!?!

In [25]:
COK12_adminUnits.tail()

19205    784
41010    678
62050    660
35030    625
49010    600
dtype: int64

In [57]:
#Let's count how many different financial values are in each column.
COK12_schoolCodes = pd.value_counts(COK12_df2['school_code'].values,sort=True)
COK12_schoolCodes.head()

0       68345
24       1163
4516      771
20        695
1426      496
dtype: int64

### Investigate why the maximum <u>0</u> calue for school_code!?!

In [27]:
COK12_schoolCodes.tail()

2966    1
1455    1
4518    1
4520    1
6009    1
dtype: int64

In [46]:
#Let's count how many different financial values are in each column.
COK12_fundCodes = pd.value_counts(COK12_df2['fund_code'].values,sort=True)
COK12_fundCodes.head()

10    156838
22     31148
11     24780
21     19035
23     16804
dtype: int64

### Investigate the top 5 <u>10</u> through <u>23</u> fiund_code!?!

In [29]:
COK12_fundCodes.tail()

51    5
26    2
58    2
59    2
75    2
dtype: int64

In [47]:
#Let's count how many different financial values are in each column.
COK12_locationCodes = pd.value_counts(COK12_df2['location_code'].values,sort=True)
COK12_locationCodes.head()

100    79496
300    39963
600    35268
200    29514
0      23168
dtype: int64

### Investigate the top <u>100</u> through <u>200</u> top location_code!?!
### Investigate why <u>0</u> is of the top 5 used location_code!?!

In [31]:
COK12_locationCodes.tail()

182    2
104    2
134    2
623    2
983    1
dtype: int64

In [48]:
#Let's count how many different financial values are in each column.
COK12_sreCodes = pd.value_counts(COK12_df2['sre_code'].values,sort=True)
COK12_sreCodes.head()

0     268470
60      3035
95      2775
21       704
22       580
dtype: int64

### Investigate why <u>0</u> is the largest used sre_code!?!

In [33]:
COK12_sreCodes.tail()

75    3
42    2
7     1
1     1
29    1
dtype: int64

In [49]:
#Let's count how many different financial values are in each column.
COK12_programCodes = pd.value_counts(COK12_df2['program_code'].values,sort=True)
COK12_programCodes.head()

0       39439
2400    21585
10      19640
2200    18478
2100    17312
dtype: int64

### Investigate why <u>0</u> is the largest used program_code!?!

In [35]:
COK12_programCodes.tail()

1969    1
1771    1
1972    1
2122    1
2161    1
dtype: int64

In [51]:
COK12_objectSourceCodes = pd.value_counts(COK12_df2['object_source_code'].values,sort=True)
COK12_objectSourceCodes.head()

100    50875
200    50215
600    33490
500    11008
580    10824
dtype: int64

### Investigate the maximum <u>100</u> through <u>580</u> top object_source_code!?!

In [37]:
COK12_objectSourceCodes.tail()

3240    1
1810    1
7441    1
618     1
0       1
dtype: int64

In [52]:
COK12_jobClassCodes = pd.value_counts(COK12_df2['job_class_code'].values,sort=True)
COK12_jobClassCodes.head()

0      172392
200     52661
400     21075
600      7722
500      7504
dtype: int64

### Investigate why <u>0</u> is the largest used job_class_code!?!

In [39]:
COK12_jobClassCodes.tail()

630    3
634    3
601    2
401    2
417    2
dtype: int64

In [53]:
COK12_grantCodes = pd.value_counts(COK12_df2['grant_code'].values,sort=True)
COK12_grantCodes.head()

0       197260
3130     11884
4010      9629
1         8084
3120      4242
dtype: int64

### Investigate why <u>0</u> is the largest used grant_code!?!

In [41]:
COK12_grantCodes.tail()

3162    1
7389    1
9005    1
1787    1
9036    1
dtype: int64

In [54]:
# Let's groupBy object_source_code codes.
COK12_groupByOSC = COK12_df2.groupby('object_source_code')
COK12_groupByOSC.head()

,district_code,admin_unit,school_code,fund_code,location_code,sre_code,program_code,object_source_code,job_class_code,grant_code,amount_cents
0,3080,64203,0,10,600,0,20,100,200,0,71250
1,3080,64203,0,10,600,0,10,100,200,4010,13841274
2,3080,64203,0,10,700,0,2700,100,100,0,12171639
3,3080,64203,0,21,700,0,3100,100,100,0,4079700
4,3080,64203,4854,10,200,0,1700,100,400,3130,8660925
...,...,...,...,...,...,...,...,...,...,...,...
156473,3148,64203,6812,10,800,0,2850,525,0,0,206878
156518,3148,64203,6812,21,700,0,3100,631,0,0,264425
156521,3148,64203,6812,21,700,0,3100,634,0,0,629105
189855,880,16010,8945,11,966,0,2700,512,0,0,1278176


### Investigate the <u>specific</u> object source codes!

In [55]:
groupFLP = COK12_df2.groupby(['fund_code', 'location_code', 'program_code'])
groupFLP.max()

district_code  admin_unit  school_code  \
fund_code location_code program_code                                           
6         0             0                      3000       59010            0   
          600           2200                   3000       59010            0   
                        2300                   3000       59010            0   
                        2800                   3000       59010            0   
          700           2600                   3000       59010            0   
...                                             ...         ...          ...   
85        971           0                       880       16010            0   
                        3300                    880       16010            0   
90        0             0                      3230       64233         9360   
          600           0                       123        3020            0   
          800           0                      3140       64193            0   

                                      sre_code  object_source_code  \
fund_code location_code program_code                                 
6         0             0                    0                8121   
          600           2200                 0                 730   
                        2300                 0                 300   
                        2800                 0                 500   
          700           2600                 0                 400   
...                                        ...                 ...   
85        971           0                    0                8242   
                        3300                 0                 800   
90        0             0                    0                7519   
          600           0                    0                7515   
          800           0                    0                7515   

                                      job_class_code  grant_code  amount_cents  
fund_code location_code program_code                                            
6         0             0                          0           0     290050022  
          600           2200                       0           0      18494795  
                        2300                       0           0        463561  
                        2800                       0           0      16749358  
          700           2600                       0           0       4800000  
...                                              ...         ...           ...  
85        971           0                          0           0    1271687000  
                        3300                       0           0     797166000  
90        0             0                          0           0   57318500000  
          600           0                          0           0    2193697300  
          800           0                          0           0  178359000000  

[6153 rows x 8 columns]

### Investigate <u>fund codes</u>, <u>location codes</u>, and <u>program codes</u>!!

In [ ]:
# To Do:
# Research code to add decimal to amount_cents into dollar type or dollar value.
# Research more on how best to aggregate and group relevent data.
# Begin adding visualizations based on the relevant financial data to present.
# Begin first phases of KNN and DBSCAN

In [ ]:
# Stopped code here

Poor attempt at paandas profile report....

In [13]:
# Generate pandas profile report
COK12_profile = ProfileReport(COK12_df, title='Pandas CO K12 Profiling Report', html={'style':{'full_width':True}})

TypeError: _plot_histogram() got an unexpected keyword argument 'title'

In [11]:
if name=="main":
    multiprocessing.freeze_support()
create_pd_profile()

NameError: name 'name' is not defined

In [ ]:
# Display profile report in Jupyter Notebook widgets
COK12_profile.to_widgets()

In [ ]:
# Include html report in Jupyter Noteboook
COK12_profile.to_notebook_iframe()

In [ ]:
# Save the profile report out to html
COK12_profile.to_file(output_file="your_report.html")